# Hypothesentests
* H01: Die Mitglieder der AfD twittern genauso viele oder weniger OFFENSE-Tweets als die Mitglieder jeder anderen Partei.
* H02: Die Häufigkeit von OFFENSE-Tweets vor dem Corona Lockdown unterscheidet sich NICHT von der während dem Lockdown.
* H03: Die Häufigkeit von OFFENSE-Tweets nach dem Corona Lockdown unterscheidet sich NICHT von der während dem Lockdown.
* H04: Die Häufigkeit von OFFENSE-Tweets vor dem Corona Lockdown unterscheidet sich NICHT von der nach dem Lockdown.
* H05: Die Mitglieder der AfD rufen im Parlament höchstens genauso oft OFFENSE-Zurufe hinein wie die Mitglieder jeder anderen Partei.
* H06: Die Mitglieder jeder Partei rufen im Plenum höchstens genauso oft OFFENSE-Zurufe hinein wie sie OFFENSE-Tweets absetzen.
* H07: Der Anteil von OFFENSE-Zurufen vor Corona ist genauso groß oder kleiner als währenddessen.


### Load Data -Twitter all

In [1]:
import pandas as pd
import datetime
import pickle, gensim, numpy as np

sent_df_Wien = pickle.load(open('/home/lisa/Darmstadt/Master Arbeit/06_Analyse/01_Twitter/sent_df_Wien_final.p', "rb" ))
# Tweets von Johannes Kahrs werdne entfernt um Normalverteilung zu erhalten
sent_df_Wien = sent_df_Wien.loc[sent_df_Wien['user_screen_name'] != 'kahrs'] 

In [ ]:
import pandas as pd
time = ['month', 'week', 'date'] # 15 Monate, 67 Wochen, 454 Tage
cat = ['', 'OFFENSE', 'OTHER']
twitter_dic = {}


for c in cat: 
    if c == '':
        sent_df = sent_df_Wien
    else:
        sent_df = sent_df_Wien.loc[sent_df_Wien['predict'] == c]
    for t in time:
        twitter_dic['piv_'+t+c] = pd.pivot_table(sent_df, index = [t], columns = ['user_party'], aggfunc = np.count_nonzero)['full_text'].fillna(0)  

twitter_dic.keys()

for t in time:
    # Normiere OFFENSE Tweets
    twitter_dic['piv_'+t+'_rel'] = (twitter_dic['piv_'+t+'OFFENSE'] / twitter_dic['piv_'+t])*100

## Check for Normality - Twitter all

In [ ]:
# https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/
# generate gaussian data
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(100) + 50

# summarize
print('mean=%.3f stdv=%.3f' % (mean(data), std(data)))
print('mean=%.3f stdv=%.3f' % (mean(twitter_dic['piv_week']['Union']), std(twitter_dic['piv_week']['Union'])))

In [ ]:
durch = []
partei = []
daten = []

for dat in twitter_dic.keys():
    for party in twitter_dic['piv_month'].columns:
        daten.append(dat)
        partei.append(party)
        durch.append('Mean: '+ str(mean(twitter_dic[dat][party]).round(3)) + ' Std:'+str(std(twitter_dic[dat][party]).round(3)))

data_mean_std = pd.DataFrame({'Datensatz': daten, 'Partei': partei, 'Mean': durch})
data_mean_std_piv = pd.pivot_table(data_mean_std, values = 'Mean', index = 'Datensatz', columns = 'Partei', aggfunc = min)

In [ ]:
data_mean_std_piv

### Scatterplots

In [ ]:
from matplotlib import pyplot
datensatz = twitter_dic['piv_month_rel']
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']

fig = pyplot.figure(figsize = (16,10))
fig.suptitle('Scatterplots der monatlichen Summe an Tweets', fontsize = 18)

num = 0
for party, n in zip(datensatz.columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.scatter(datensatz.index, datensatz[party], color = my_palette[num])
    x_tags = pyplot.xticks(datensatz.index, fontsize = 7, rotation = 45, ha = "right")
    ax.set_title(party)
    num = num + 1

### Histograme

In [ ]:
datensatz = twitter_dic['piv_week_rel']
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']
Parteien = ['AFD', 'FDP', 'Gruene', 'Linke', 'SPD', 'Union']

fig = pyplot.figure(figsize = (13,8))
fig.suptitle('Histogramme der relativen Häufigkeit von OFFENSE-Tweets pro Woche', fontsize = 18)

num = 0
for party, n in zip(datensatz.columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.hist(datensatz[party], color = my_palette[num])
    ax.set_title(party)
    num = num + 1


### QQ Plot

In [ ]:
# QQ Plot
from statsmodels.graphics.gofplots import qqplot

qqplot(twitter_dic['piv_week_rel']['SPD'], line='s')
pyplot.show()

### Shapiro-Wilk
Ergebnis: 

* Die meisten sind nicht Normalverteilt - überraschend, garnicht bei piv_date Datensätzen! -> Ergibt Sinn, weil Shapiro Wilk nur für < 50 Beobachtungen!
* piv_month und piv_monthOTHER komplett Normalverteilt - Union, Linke und AFD auch bei OFFENSE Tweets
* piv_week und piv_weekOTHER ebenfalls komplett - Nur AfD auch bei OFFENSE Tweets normalverteilt!

In [ ]:
# Shapiro-Wilk Test
from scipy.stats import shapiro

partei = []
dataset = []
result = []

for party in twitter_dic['piv_month'].columns:
    for dat in twitter_dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test
        stat, p = shapiro(twitter_dic[dat][party])

# interpret
        alpha = 0.05
        if p > alpha:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' Gaussian')
        else:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' NOT Gaussian')
            
Shapiro_Wilk_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
Shapiro_Wilk_piv = pd.pivot_table(Shapiro_Wilk_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max)
Shapiro_Wilk_piv

## Corellationstests - Twitter all

### Spearman’s Rank Correlation
-> Pearson würde zu gleichen Ergebnisse führen, aber Varianze nicht gleich und nur teilweise Normalverteilt (Verletzt Voraussetzungen)!

*   Observations in each sample are independent and identically distributed (iid).
*   Observations in each sample can be ranked.


*   H0: the two samples are independent.
*   H1: there is a dependency between the samples.

Ergibt Sinn, dass fasst alle dependent sind! -> Getrieben von äußeren Events auf die fast alle Politiker*innen reagieren

In [ ]:
from scipy.stats import spearmanr

partei = []
dataset = []
result = []

for party in twitter_dic['piv_month'].columns:
    for dat in twitter_dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test
        stat, p = spearmanr(twitter_dic[dat][party], twitter_dic[dat]['AFD'])

        if p > 0.05:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + 'INDEPENDENT')
        else:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + 'Probably dependent')
            
Corr_Spearman_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
Corr_Spearman_piv =  pd.pivot_table(Corr_Spearman_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max) 
# Ergibt Sinn, dass ALLE dependent sind!
# -> Getrieben von äußeren Events auf die fast alle Politiker*innen reagieren
Corr_Spearman_piv

### Chi-Squared Test

Tests whether two categorical variables are related or independent.

*   Observations used in the calculation of the contingency table are independent.
*   25 or more examples in each cell of the contingency table.

-> Es gibt eine Abhängigkeit zwischen Partei und Häufigkeit von OFFENSE/ OTHER Tweets (Bei Tests zwischen einzelnen Parteien nicht immer vorhanden, z.B Grüne und Linke sind dependent, SPD und FDP nicht!)

In [ ]:
contingency_table = pd.pivot_table(sent_df_Wien, index = 'user_party', columns = 'predict', aggfunc=np.count_nonzero)['full_text']
# sent_df_Wien_noAfD = sent_df_Wien.loc[(sent_df_Wien['user_party'] == 'Gruene')|(sent_df_Wien['user_party'] == 'Linke')]

In [ ]:
from scipy.stats import chi2_contingency
stat, p, dof, expected = chi2_contingency(contingency_table)
print('stat=%.3f, p=%.3f' % (stat, p))

if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

# H01: Die Mitglieder der AfD twittern genauso viele oder weniger OFFENSE-Tweets als die Mitglieder jeder anderen Partei.
* Zur beantwortung der Frage werden 

### Student’s t-test

Tests whether the means of two independent samples are significantly different.

*    Observations in each sample are independent and identically distributed (iid).
*    Observations in each sample are normally distributed.
*    Observations in each sample have the same variance.

Interpretation von stats.ttest_ind(B,A)

*    H0: A >= B (AFD hat insgesamt mehr Tweets als anderen Parteien)
*    H1: B > A  (AFD hat weniger Tweets als andere Parteien)

In [ ]:
from scipy.stats import ttest_ind

partei = []
dataset = []
result = []

for party in twitter_dic['piv_month_rel'].columns:
    for dat in twitter_dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test
        stat, p = ttest_ind(twitter_dic[dat]['AFD'], (twitter_dic[dat][party]))
        if p/2 > 0.05 or stat < 0:    # p/2 für Einseitigen t-Test
            result.append('stat=%.3f, p=%.3f' % (stat, p) + party +' gleich oder mehr als AfD' )
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + party +' hat WENIGER als AfD')
                        
H01_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
H01_piv =  pd.pivot_table(H01_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max) 
H01_piv

### ANOVA

In [ ]:
from scipy.stats import f_oneway
dataset = twitter_dic['piv_week_rel']

stat, p = f_oneway(dataset['AFD'], dataset['FDP'], dataset['Union'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably the same distribution')
else:
    print('Probably different distributions')

## Nichtparametrische - Mann-Whitney U Test

In [ ]:
# Example of the Mann-Whitney U Test
from scipy.stats import mannwhitneyu

partei = []
dataset = []
result = []

for party in twitter_dic['piv_month_rel'].columns:
    for dat in twitter_dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test
        stat, p = mannwhitneyu(twitter_dic[dat]['AFD'], twitter_dic[dat][party])
        if p > 0.05:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + party +' gleich oder weniger als AfD' )
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + party +' AfD hat mehr')

          
Mann_Whitney_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
Mann_Whitney_piv =  pd.pivot_table(Mann_Whitney_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max) 
Mann_Whitney_piv

In [ ]:
import numpy as np
import pingouin as pg
from pingouin import ttest
#https://pingouin-stats.org/generated/pingouin.mwu.html
#https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest


for party in twitter_dic['piv_month_rel'].columns:
    dat = twitter_dic['piv_week_rel']
    partei.append(party)
    dataset.append(dat)
# normality test
    print(party)
    print(mannwhitneyu(dat['AFD'], dat[party]))
    print(pg.mwu(dat[party], dat['AFD']))
    #print(ttest(dat['AFD'], dat[party], paired=True, tail='greater').round(2))
        


# H02: Die Häufigkeit von OFFENSE-Tweets vor dem Corona Lockdown unterscheidet sich NICHT von der während des Lockdowns.

=> WICHTIG: Für H02-H04 muss beachtet werden, dass es sich um verbundene Stichproben handelt!

In [ ]:
# Eingrenzen der relevanten Zeiträume
sent_Corona = sent_df_Wien.loc[(sent_df_Wien['date'] >= datetime.date(2020,3,13))
                & (sent_df_Wien['date'] < datetime.date(2020,5,6))]

sent_before_Corona = sent_df_Wien.loc[(sent_df_Wien['date'] < datetime.date(2020,3,13))]
sent_after_Corona = sent_df_Wien.loc[(sent_df_Wien['date'] >= datetime.date(2020,5,6))]

In [ ]:
# Erstelle Dictionarys für Zeträume
import pandas as pd

Corona_dic = {}
before_Corona_dic = {}
after_Corona_dic = {}

dics =[Corona_dic, before_Corona_dic, after_Corona_dic]
datasets = [sent_Corona, sent_before_Corona, sent_after_Corona]

for dataset, dic in zip(datasets, dics):
    time = ['month', 'week', 'date']
    cat = ['', 'OFFENSE', 'OTHER']
    
    for c in cat: 
        if c == '':
            sent_df = dataset
        else:
            sent_df = dataset.loc[dataset['predict'] == c]
        for t in time:
            dic['piv_'+t+c] = pd.pivot_table(sent_df, index = [t], columns = ['user_party'], aggfunc = np.count_nonzero)['full_text'].fillna(0)         
            
for t in time:
    for dic in dics:
        # Normiere OFFENSE Tweets
        dic['piv_'+t+'_rel'] = (dic['piv_'+t+'OFFENSE'] / dic['piv_'+t])*100

## Check for Normality

In [ ]:
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std

durch = []
abw = []
partei = []
daten = []
dic = before_Corona_dic

for dat, n in zip(dic, dic.keys()):
    for party in dic['piv_month'].columns:
        daten.append(n)
        partei.append(party)
        durch.append(mean(dic[dat][party]).round())
        abw.append(std(dic[dat][party]))

mean_std = pd.DataFrame({'Datensatz': daten, 'Partei': partei, 'Mean': durch, 'Std': abw})
#mean_std

### Scatterplots

In [ ]:
from matplotlib import pyplot
datensatz = after_Corona_dic['piv_weekOFFENSE']
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']

fig = pyplot.figure(figsize = (16,10))
fig.suptitle('Scatterplots der monatlichen Summe an Tweets', fontsize = 18)

num = 0
for party, n in zip(Corona_dic['piv_date'].columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.scatter(datensatz.index, datensatz[party], color = my_palette[num])
    x_tags = pyplot.xticks(datensatz.index, fontsize = 7, rotation = 45, ha = "right")
    ax.set_title(party)
    num = num + 1

### Histograme

In [ ]:
datensatz =  after_Corona_dic['piv_date_rel']
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']
Parteien = ['AFD', 'FDP', 'Gruene', 'Linke', 'SPD', 'Union']

fig = pyplot.figure(figsize = (16,10))
fig.suptitle('Histogramme der Summe an Zurufen', fontsize = 18)

num = 0
for party, n in zip(Corona_dic['piv_week'].columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.hist(datensatz[party], color = my_palette[num])
    ax.set_title(party)
    num = num + 1


### Shapiro-Wilk

In [ ]:
from scipy.stats import shapiro

partei = []
dataset = []
result = []
dic = after_Corona_dic

for party in Corona_dic['piv_date'].columns:
    for dat in dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test H0: Dataset frwan from Gausian distribution
        stat, p = shapiro(dic[dat][party])

        alpha = 0.05
        if p > alpha:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' Gaussian') # Weise H0 nicht zurück
        else:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' NOT Gaussian')
            
Shapiro_Wilk_Corona_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
Shapiro_Wilk_Corona_piv = pd.pivot_table(Shapiro_Wilk_Corona_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max)
Shapiro_Wilk_Corona_piv

In [ ]:
contingency_Corona = pd.pivot_table(sent_after_Corona, index = 'user_party', columns = 'predict', aggfunc=np.count_nonzero)['full_text']
contingency_Corona

In [ ]:
from scipy.stats import chi2_contingency
stat, p, dof, expected = chi2_contingency(contingency_Corona)
print('stat=%.3f, p=%.3f' % (stat, p))

if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')
# Häufigkeit OFFENSE oder OTHER ist abhängig von der Partei 

In [ ]:
## T-Test eigentlich schlecht - weil nicht normalverteilt!

from scipy.stats import ttest_ind
partei = []
result = []    

for party in before_Corona_rel_piv.columns:
        partei.append(party)
# normality test
        stat, p = ttest_ind(before_Corona_rel_piv[party], Corona_rel_piv[party])
        if p/2 > 0.05 or stat < 0:    # p/2 für Einseitigen t-Test
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleich oder mehr als vor dem Lockdown' )
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' WENIGER als vor dem Lockdown')
          
                        
H03_bef_Corona_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
H03_bef_Corona_df

## Wilcoxon Rang Test

Da getestet werden soll, ob vor- bzw. nach dem Lockdown MEHR OFFENSE-Tweets abgesetzt wurden, werden einseitige Tests durchgeführt!

ABER: Es wird nicht die option "alternative = 'greater'" (scipy) oder "tail='greater'" (pinguin) verwendet! Beide passen den p-Wert automatisch an und zeigen die aufsummierten Ränge von der größeren Statistik. Das entspricht nicht den gängigen Anleitungen, nach denen der kleinere Wert der Statistik unter einem Schwellenwert liegen muss. Der p Wert wird also "manuell" halbiert und mit dem Ergebnis verglichen

In [ ]:
# Example of the Mann-Whitney U Test
from scipy.stats import wilcoxon

partei = []
result = []

for party in before_Corona_rel_piv.columns:
        partei.append(party)
# normality test
        stat, p = wilcoxon(before_Corona_dic['piv_week_rel'][38:][party], Corona_dic['piv_week_rel'][party])
        #stat, p = wilcoxon(before_Corona_dic['piv_date_rel'][260:][party], Corona_dic['piv_date_rel'][party])
        if p/2 > 0.05:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleich oder mehr als vor dem Lockdown')
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' WENIGER als vor dem Lockdown')

          
wilcoxon_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
wilcoxon_df

In [ ]:
import numpy as np
import pingouin as pg
from pingouin import ttest
#https://pingouin-stats.org/generated/pingouin.mwu.html
#https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

for party in before_Corona_dic['piv_date_rel'].columns:
# normality test
        #stat, p = wilcoxon(before_Corona_dic['piv_week_rel'][38:][party], Corona_dic['piv_week_rel'][party])
        print(party)
        #print(pg.wilcoxon(before_Corona_dic['piv_date_rel'][260:][party], Corona_dic['piv_date_rel'][party]))
        print(pg.wilcoxon(before_Corona_dic['piv_week_rel'][38:][party], Corona_dic['piv_week_rel'][party]))

In [ ]:
before_Corona_dic['piv_week_rel'][38:]
#before_Corona_dic['piv_week_rel']

In [ ]:
# Randomisierte ziehung der neun Wochen aus den Wochen vor dem Lockdown
import random

sample_before = random.sample(list(before_Corona_dic['piv_week_rel'].index), 9)
before_week_sample = before_Corona_dic['piv_week_rel'].loc[sample_before]

for party in before_Corona_dic['piv_date_rel'].columns:
# normality test
        #stat, p = wilcoxon(before_Corona_dic['piv_week_rel'][38:][party], Corona_dic['piv_week_rel'][party])
        print(party)
        #print(pg.wilcoxon(before_Corona_dic['piv_date_rel'][260:][party], Corona_dic['piv_date_rel'][party]))
        print(pg.wilcoxon(before_week_sample[party], Corona_dic['piv_week_rel'][party]))

# H03: Die Häufigkeit von OFFENSE-Tweets während dem Corona Lockdown unterscheidet sich NICHT von der nach dem Lockdown.

In [ ]:
## T-Test eigentlich schlecht - weil nicht normalverteilt!

from scipy.stats import ttest_ind
partei = []
result = []    

for party in after_Corona_rel_piv.columns:
        partei.append(party)
# normality test
        stat, p = ttest_ind(after_Corona_rel_piv[party], Corona_rel_piv[party])
        if p/2 > 0.05 or stat < 0:    # p/2 für Einseitigen t-Test
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleich oder mehr als nach dem Lockdown' )
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' WENIGER als nach dem Lockdown')
          
                        
H03_bef_Corona_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
H03_bef_Corona_df

In [ ]:
# Example of the Mann-Whitney U Test
from scipy.stats import wilcoxon

partei = []
result = []

for party in before_Corona_rel_piv.columns:
        partei.append(party)
# normality test
        stat, p = wilcoxon(after_Corona_dic['piv_week_rel'][:9][party], Corona_dic['piv_week_rel'][party])
        #stat, p = wilcoxon(after_Corona_dic['piv_date_rel'][:54][party], Corona_dic['piv_date_rel'][party])
        if p/2 > 0.05:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleich oder mehr als nach dem Lockdown')
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' WENIGER als nach dem Lockdown')

          
wilcoxon_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
wilcoxon_df

In [ ]:
import numpy as np
import pingouin as pg
from pingouin import ttest
#https://pingouin-stats.org/generated/pingouin.mwu.html
#https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

for party in before_Corona_dic['piv_date_rel'].columns:
# normality test
        #stat, p = wilcoxon(before_Corona_dic['piv_week_rel'][38:][party], Corona_dic['piv_week_rel'][party])
        print(party)
        #print(pg.wilcoxon(after_Corona_dic['piv_date_rel'][:54][party], Corona_dic['piv_date_rel'][party]))
        print(pg.wilcoxon(after_Corona_dic['piv_week_rel'][:9][party], Corona_dic['piv_week_rel'][party]))

# H04: Die Häufigkeit von OFFENSE-Tweets vor dem Corona Lockdown unterscheidet sich NICHT von der nach dem Lockdown.

In [ ]:
before_Corona_rel_piv = (before_Corona_dic['piv_dateOFFENSE'] / before_Corona_dic['piv_date'])*100
after_Corona_rel_piv = ((after_Corona_dic['piv_dateOFFENSE']/ after_Corona_dic['piv_date'])*100).fillna(0)
bef_Corona_rel_piv = before_Corona_rel_piv[228:]

In [ ]:
len(before_Corona_rel_piv) - len(after_Corona_rel_piv)
len(before_Corona_rel_piv[228:])

In [ ]:
from scipy.stats import ttest_rel
partei = []
result = []    

for party in after_Corona_rel_piv.columns:
        partei.append(party)
# normality test
        stat, p = ttest_rel(after_Corona_rel_piv[party], bef_Corona_rel_piv[party])
        if p > 0.05:    
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleiche Häufigkeit vor und nach dem Lockdown' )
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Andere Häufigkeiten vor und nach dem Lockdown')
          
                        
H04_Corona_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
H04_Corona_df

In [ ]:
from scipy.stats import wilcoxon

partei = []
result = []

for party in before_Corona_rel_piv.columns:
        partei.append(party)
# normality test
        stat, p = wilcoxon(after_Corona_dic['piv_week_rel'][party], before_Corona_dic['piv_week_rel'][34:][party])
        #stat, p = wilcoxon(after_Corona_dic['piv_date_rel'][party], before_Corona_dic['piv_date_rel'][228:][party])
        if p/2 > 0.05:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Davor weniger oder gleich')
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Davor MEHR')

          
wilcoxon_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
wilcoxon_df

In [ ]:
import numpy as np
import pingouin as pg
from pingouin import ttest
#https://pingouin-stats.org/generated/pingouin.mwu.html
#https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

for party in before_Corona_dic['piv_date_rel'].columns:
# normality test
        #stat, p = wilcoxon(before_Corona_dic['piv_week_rel'][38:][party], Corona_dic['piv_week_rel'][party])
        print(party)
        #print(pg.wilcoxon(after_Corona_dic['piv_date_rel'][party], before_Corona_dic['piv_date_rel'][228:][party], tail='less'))
        print(pg.wilcoxon(before_Corona_dic['piv_week_rel'][34:][party], after_Corona_dic['piv_week_rel'][party]))

# Plenum

In [ ]:
import pickle, gensim, numpy as np
import pandas as pd 

#df_kom = mdp_partei_emo
PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/'
#pickle.dump(df_kom, open(PICKLE_FOLDER_PATH + 'Plenum_Kom_all_Sent', "wb" ))

df_kom = pickle.load(open(PICKLE_FOLDER_PATH + 'Plenum_Kom_all_Sent', "rb" ))

In [ ]:
import pandas as pd
time = ['Monat', 'Woche', 'date_str']
cat = ['', 'OFFENSE', 'OTHER']
plenum_dic = {}


for c in cat: 
    if c == '':
        sent_df = df_kom
    else:
        sent_df = df_kom.loc[df_kom['predict'] == c]
    for t in time: 
        piv = pd.pivot_table(sent_df, index = [t], columns = ['Partei'], aggfunc = np.count_nonzero)['Beschreibung1'].fillna(0)
        
        plenum_dic['piv_'+t+c] =  piv       
            
for woche in ['17-w43', '18-w03', '18-w36', '19-w29', '19-w36']:
    plenum_dic['piv_Woche'] = plenum_dic['piv_Woche'].loc[plenum_dic['piv_Woche'].index != woche]
    plenum_dic['piv_WocheOFFENSE'] = plenum_dic['piv_WocheOFFENSE'].loc[plenum_dic['piv_WocheOFFENSE'].index != woche]

# Normiere OFFENSE Tweets
plenum_dic['piv_Woche_rel'] = (plenum_dic['piv_WocheOFFENSE'] / plenum_dic['piv_Woche'])*100
    
plenum_dic.keys()

# Check for Normality

### Scatterplots

In [ ]:
from matplotlib import pyplot
datensatz = plenum_dic['piv_Woche_rel']
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']

fig = pyplot.figure(figsize = (16,10))
fig.suptitle('Scatterplots der monatlichen Summe an Tweets', fontsize = 18)

num = 0
for party, n in zip(plenum_dic['piv_Woche_rel'].columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.scatter(datensatz.index, datensatz[party], color = my_palette[num])
    x_tags = pyplot.xticks(datensatz.index, fontsize = 7, rotation = 45, ha = "right")
    ax.set_title(party)
    num = num + 1

### Histograme

In [ ]:
datensatz =  plenum_dic['piv_Woche_rel']
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'green', 'black', 'purple', 'orange', 'red']

fig = pyplot.figure(figsize = (13,8))
fig.suptitle('Histogramme der Summe an Zurufen', fontsize = 18)

num = 0
for party, n in zip(plenum_dic['piv_Woche_rel'].columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.hist(datensatz[party], color = my_palette[num])
    ax.set_title(party)
    num = num + 1


In [ ]:
from numpy import mean
from numpy import std
partei = []
durch = []
abw = []

for party in plenum_dic['piv_Woche_rel'].columns:
        partei.append(party)
        durch.append('Mean: '+ str(mean(plenum_dic['piv_Woche_rel'][party]).round(3)))
        abw.append('Std:'+str(std(plenum_dic['piv_Woche_rel'][party]).round(3)))
        
plenum_data_mean_std = pd.DataFrame({'Partei': partei, 'Mean': durch, 'Std': abw})
plenum_data_mean_std

### Shapiro-Wilk

In [ ]:
from scipy.stats import shapiro

partei = []
dataset = []
result = []

for party in plenum_dic['piv_date_str'].columns:
    for dat in plenum_dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test
        stat, p = shapiro(plenum_dic[dat][party])

        alpha = 0.05
        if p > alpha:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' Gaussian')
        else:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' NOT Gaussian')
            
Shapiro_Wilk_plen_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
Shapiro_Wilk_plen_piv = pd.pivot_table(Shapiro_Wilk_plen_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max)
Shapiro_Wilk_plen_piv

In [ ]:
contingency_plenum = pd.pivot_table(df_kom, index = 'Partei', columns = 'predict', aggfunc=np.count_nonzero)['Beschreibung1']

In [ ]:
from scipy.stats import chi2_contingency
stat, p, dof, expected = chi2_contingency(contingency_plenum)
print('stat=%.3f, p=%.3f' % (stat, p))

if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

# H05: Die Mitglieder der AfD rufen im Parlament höchstens genauso oft OFFENSE-Zurufe hinein wie die Mitglieder jeder anderen Partei.

Für diese Hypothese wird noch der komplette Datensatz verwendet - also alle Wochen aus der betrachteten Legislaturperiode. Bei H06 hingegen, werden Twitter und Plenardaten verglichen. Dafür werden die Wochen in denen Plenarsitzungen stattgefunden haben ab Woche 19-w18, auf Twitter und im Plenum betrachtet.

In [ ]:
# EMann-Whitney für alle Datensätze (Woche, Monat, Tag, OFFENSE, OTHER, ...)
from scipy.stats import mannwhitneyu

partei = []
dataset = []
result = []

for party in plenum_dic['piv_date_str'].columns:
    for dat in plenum_dic.keys():
        partei.append(party)
        dataset.append(dat)
# normality test
        stat, p = mannwhitneyu(plenum_dic[dat]['AfD'], plenum_dic[dat][party])
        if p/2 > 0.05:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleich oder mehr als AfD' )
        else:
            result.append('stat=%.3f, p=%.3f' % (stat, p) + ' WENIGER als AfD')

          
Mann_Whitney_df = pd.DataFrame({'Partei':partei, 'Daten':dataset, 'Ergebnis':result})
Mann_Whitney_piv =  pd.pivot_table(Mann_Whitney_df, values='Ergebnis', index=['Daten'], columns=['Partei'], aggfunc=max) 
#Mann_Whitney_piv

In [ ]:
# Ergebnisse Mann_Whitney für relative OFFENSE-Zurufe (Woche)
# Hinweis - es wurden alle NaNs im Voraus entfernt und ein Extremwert bei der AfD (100% OFFENSE) 
# -> andernfalls ergeben sich kleine Unterschiede zwischen zwischen scipy und pingu
from scipy.stats import mannwhitneyu

partei = []
dataset = []
result = []

for party in plenum_dic['piv_date_str'].columns:
    for dat in ['piv_Woche_rel']:
        print(party)
# normality test
        print(mannwhitneyu(plenum_dic[dat]['AfD'], plenum_dic[dat][party]))
        print(pg.mwu(plenum_dic[dat][party], plenum_dic[dat]['AfD']))


In [ ]:
len(plenum_dic['piv_Woche_rel']['AfD'])

# H06: Die Mitglieder jeder Partei rufen im Plenum höchstens genauso oft OFFENSE-Zurufe hinein wie sie OFFENSE-Tweets absetzen.

Es werden Tweets und Zurufe auf Wochenbasis miteinander verglichen. Die Sitzungen im Bundestag haben zwar selten das selbe Thema, das aktuell auf Twitter aktuell ist, aber potenziell sind Stimmungen der Abgeordneten trotzdem auf beiden Kanälen zu erkennen.

In [ ]:
# Wähle Wochen auf Twitter aus, die auch Sitzungswochen waren
twitter_OF = twitter_dic['piv_weekOFFENSE'].loc[list(plenum_dic['piv_WocheOFFENSE'].index[30:])]
twitter = twitter_dic['piv_week'].loc[list(plenum_dic['piv_Woche'].index[30:])]
twitter_rel_piv = (twitter_OF/twitter)*100

plenum_rel_piv = ((plenum_dic['piv_WocheOFFENSE'][30:] / plenum_dic['piv_Woche'][30:])*100).fillna(0)

In [ ]:
#datensatz = plenum_rel_piv
datensatz = twitter_rel_piv
pyplot.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']
#my_palette = ['blue', 'green', 'black', 'purple', 'orange', 'red']
Parteien = ['AFD', 'FDP', 'Gruene', 'Linke', 'SPD', 'Union']

fig = pyplot.figure(figsize = (13,8))
fig.suptitle('Histogramme der täglichen Summe an Tweets', fontsize = 18)

num = 0
for party, n in zip(datensatz.columns, range(1,7)):
    ax = fig.add_subplot(2,3,n)
    ax.hist(datensatz[party], color = my_palette[num])
    ax.set_title(party)
    num = num + 1

In [ ]:
from scipy.stats import shapiro
dataset = plenum_rel_piv
partei = []
result = []

for party in dataset.columns:
        partei.append(party)
# normality test
        stat, p = shapiro(dataset[party])

        alpha = 0.05
        if p > alpha:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' Gaussian')
        else:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' NOT Gaussian')
            
Shapiro_Wilk_plen_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
Shapiro_Wilk_plen_df

In [ ]:
from numpy import mean
from numpy import std

dataset = plenum_rel_piv
partei = []
durch = []
abw = []

for party in dataset.columns:
        partei.append(party)
        durch.append('Mean: '+ str(mean(dataset[party]).round(3)))
        abw.append('Std:'+str(std(dataset[party]).round(3)))
        
twitter_data_mean_std = pd.DataFrame({'Partei': partei, 'Mean': durch, 'Std': abw})
twitter_data_mean_std

In [ ]:
partei = []
durch = []
abw = []

for party in plenum_rel_piv.columns:
        partei.append(party)
        durch.append('Mean: '+ str(mean(plenum_rel_piv[party]).round(3)))
        abw.append('Std:'+str(std(plenum_rel_piv[party]).round(3)))
        
plenum_data_mean_std = pd.DataFrame({'Partei': partei, 'Mean': durch, 'Std': abw})
plenum_data_mean_std

### Spearman’s Rank Correlation


*   Observations in each sample are independent and identically distributed (iid).
*   Observations in each sample can be ranked.

Für den Vergleich von Twitter und Plenardaten werden zwei verschiedene Zeiteinheiten verwendet und die Zeiträume wurden weitgehend aneinander angeglichen - eine genaue Übereinstimmung gibt es nicht, ist jedoch auch nicht sinnvoll weil Themen im Plenum keinen tagesaktuellen Bezug haben:

* Twitter - Wochen
* Plenum - Tage

Bei den Plenardaten müssen außerdem sechs Tage entfernt werden an denen es keine OFFENSE Tweets gab - andernfalls würden nicht die gleichen Tage mit OFFENSE Tweets durch alle Tweets geteilt.

In [ ]:
from scipy.stats import spearmanr
from scipy.stats import pearsonr
#-> Es können technsich beide verwendet werden - nur unten Fkt. anpassen 
# Aber Spearman ist sinnvoller weil Annahme der Normalverteilung für piv_date_str nicht haltbar ist

partei = []
partei_plen = []
dataset = []
result = []

for party in twitter_dic['piv_month'].columns:
    for party_plen in plenum_dic['piv_date_str'].columns:
        partei.append(party)
        partei_plen.append(party_plen)
# normality test
        stat, p = spearmanr(twitter_rel_piv[party], plenum_rel_piv[party_plen])

        if p > 0.05:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + ' INDEPENDENT')
        else:
            result.append('Stats=%.3f, p=%.3f' % (stat, p) + 'Probably dependent')
            
Corr_parties_df = pd.DataFrame({'Partei':partei, 'Partei_Plenum':partei_plen, 'Ergebnis':result})
Corr_parties_piv =  pd.pivot_table(Corr_parties_df, values='Ergebnis', index=['Partei_Plenum'], columns=['Partei'], aggfunc=max) 
Corr_parties_piv
# -> Die Zurufe die im Parlament an den jeweiligen Sitzungstagen erfolgen sind unabbhängig von den Tweets innerhalb einer Woche 

In [ ]:
from scipy.stats import mannwhitneyu

plenum_rel_piv.columns = ['AFD', 'Gruene', 'Union', 'Linke', 'FDP', 'SPD']

partei = []
partei_plen = []
dataset = []
result = []   

for party in plenum_rel_piv.columns:
    partei.append(party)
# normality test
    stat, p = mannwhitneyu(twitter_rel_piv[party], plenum_rel_piv[party])
    if p > 0.05 :    # p/2 für Einseitigen t-Test
        result.append('stat=%.3f, p=%.3f' % (stat, p) + ' Gleich offensiv oder mehr im Plenum')
    else:
        result.append('stat=%.3f, p=%.3f' % (stat, p) + ' WENIGER OFFENSIV im Plenum')
                        
H06_df = pd.DataFrame({'Partei':partei, 'Ergebnis':result})
#H06_df

In [ ]:
import numpy as np
import pingouin as pg
from pingouin import ttest
#https://pingouin-stats.org/generated/pingouin.mwu.html
#https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

for party in plenum_rel_piv.columns:
# normality test
    print(party)
    print(mannwhitneyu(twitter_rel_piv[party], plenum_rel_piv[party]))
    print(pg.mwu(plenum_rel_piv[party],twitter_rel_piv[party]))
    #print(pg.mwu(plenum_rel_piv[party],twitter_rel_piv[party], tail = 'less'))

# H07: Der Anteil von OFFENSE-Zurufen vor Corona ist genauso groß oder kleiner als währenddessen.

In [ ]:
import pickle, gensim, numpy as np
import pandas as pd 

#df_kom = mdp_partei_emo
PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/'
#pickle.dump(df_kom, open(PICKLE_FOLDER_PATH + 'Plenum_Kom_all_Sent', "wb" ))

df_kom = pickle.load(open(PICKLE_FOLDER_PATH + 'Plenum_Kom_all_Sent', "rb" ))

In [ ]:
import pandas as pd
time = ['Monat', 'Woche', 'date_str']
cat = ['', 'OFFENSE', 'OTHER']
plenum_dic = {}


for c in cat: 
    if c == '':
        sent_df = df_kom
    else:
        sent_df = df_kom.loc[df_kom['predict'] == c]
    for t in time: 
        piv = pd.pivot_table(sent_df, index = [t], columns = ['Partei'], aggfunc = np.count_nonzero)['Beschreibung1'].fillna(0)
        
        plenum_dic['piv_'+t+c] =  piv       
            
for woche in ['17-w43', '18-w03', '18-w36', '19-w29', '19-w36']:
    plenum_dic['piv_Woche'] = plenum_dic['piv_Woche'].loc[plenum_dic['piv_Woche'].index != woche]
    plenum_dic['piv_WocheOFFENSE'] = plenum_dic['piv_WocheOFFENSE'].loc[plenum_dic['piv_WocheOFFENSE'].index != woche]

# Normiere OFFENSE Tweets
plenum_dic['piv_Woche_rel'] = (plenum_dic['piv_WocheOFFENSE'] / plenum_dic['piv_Woche'])*100
    
plenum_dic.keys()

In [ ]:
# Lösche Datum für das es keine OFFENSE Zurufe gab
plenum_dic['piv_date_str'] = plenum_dic['piv_date_str'].loc[plenum_dic['piv_date_str'].index != '2020-01-15']

# Nur Lockdown - wären gerade sechs Beobachtungen
#piv_Corona_plen_rel = (plenum_dic['piv_date_strOFFENSE'][146:152]/plenum_dic['piv_date_str'][151:157])*100
#piv_before_Corona_plen_rel = (plenum_dic['piv_date_strOFFENSE'][141:147]/plenum_dic['piv_date_str'][146:152])*100

# Ab Lockdown bis zur Sommerpause - 18 Beobachtungen
piv_Corona_plen_rel = (plenum_dic['piv_date_strOFFENSE'][146:]/plenum_dic['piv_date_str'][151:])*100
piv_before_Corona_plen_rel = (plenum_dic['piv_date_strOFFENSE'][128:146]/plenum_dic['piv_date_str'][133:151])*100

In [ ]:
from scipy.stats import wilcoxon

import numpy as np
import pingouin as pg
from pingouin import ttest
#https://pingouin-stats.org/generated/pingouin.mwu.html
#https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

for party in piv_Corona_plen_rel.columns:
# normality test
        print(party)
        print(wilcoxon(piv_before_Corona_plen_rel[party], piv_Corona_plen_rel[party]))
        print(pg.wilcoxon(piv_before_Corona_plen_rel[party], piv_Corona_plen_rel[party]))